# LCOH Mix

## Calling libraries

In [ ]:
import pandas as pd                         ## Manipulation des tables de données et listes
import matplotlib.pyplot as plt
import plotly.express as px
import time
import numpy as np


from scipy.optimize import minimize         ## solver Objective function
from scipy.optimize import shgo    

import datetime as dt                       ## Index dataframe to date+houar

#import plotly.express as px                 ## Plot figures
import seaborn as sns                       ## Style Figures
# import dash                                        ## Application Interface
# from jupyter_dash import JupyterDash
# from dash import dcc
# from dash import html
# import dash_bootstrap_components as dbc
# from dash.dependencies import Input, Output, State

## Wind and PV production to dataframe

In [ ]:
def prepare_AEP_data(wind_file):
    # read wind data and extract date and time
    wind_data = pd.read_csv(wind_file, delimiter="\t")
    wind_data[['Date', 'Time']] = wind_data['Date/Time'].str.split(expand=True)
    wind_data = wind_data.drop('Date/Time', axis=1)
    wind_data = wind_data.rename(columns={'Nador Wind [516 MW]': 'Nador Wind [516 MW]'})
    wind_data = wind_data.rename(columns={'Nador PV [300 MW]': 'Nador PV [300 MW]'})
    wind_data['Nador Wind [516 MW]'] = pd.to_numeric(wind_data['Nador Wind [516 MW]'].str.replace(',', '.'))
    wind_data['Nador PV [300 MW]'] = pd.to_numeric(wind_data['Nador PV [300 MW]'].str.replace(',', '.'))

    AEP_data = wind_data.set_index(['Date', 'Time'])
    
    return AEP_data
AEP_Data = prepare_AEP_data("Nador_Wind_PV.txt")
AEP_Data["Nador PV [300 MW]"]=AEP_Data["Nador PV [300 MW]"]*(1-0.1)
AEP_Data.loc[(AEP_Data["Nador PV [300 MW]"] < 0) ,"Nador PV [300 MW]"] =  0
              
AEP_Data

In [ ]:
def get_min_LCOH_nador(capex_w, opex_w, minpw, maxpw, initial_w,
                 capex_pv, opex_pv, minpv, maxpv, initial_pv,
                 capex_bess,opex_bess,minbess,maxbess,initial_bess,
                 capex_electrolyse, opex_electrolyse, conso_spécifique, maxelec, minelec, initial_elec,
                 wacc,T,
                 H2_input_tonne):
    
    def BESS(AEP_Data,C_electrolyse,C_bess,eta_ch,eta_disch):
        #### Etat de la batterie à C_electrolyse à l'instant t ############################################
        SOC_t = np.zeros(len(AEP_Data))
        SOC_t[0] = 0

        W_PV = np.array(AEP_Data["Energy Hybrid"].to_list())
        
        AEP_Data["needed (-)/ surplus(+)"] = AEP_Data["Energy Hybrid"] - float(C_electrolyse)
        
        Needed_Surplus = np.array(AEP_Data["needed (-)/ surplus(+)"].to_list())

        for i in range(len(AEP_Data)):    ##Update SOC
            if W_PV[i] >= C_electrolyse:   ###charge
                SOC_t[i] = min(C_bess, SOC_t[i-1] + Needed_Surplus[i]*eta_ch)  
            else:   ## discharge
                SOC_t[i] = max(0, SOC_t[i-1] + Needed_Surplus[i]*(1/(eta_disch)))
        AEP_Data["SOC(t)"] =  SOC_t

        ########################################################################################
        #### Energie nette produite 
        AEP_Data["Energy Bess"] = ""
        AEP_Data["Energy Bess"] = (AEP_Data["SOC(t)"] - AEP_Data.shift(periods=1, fill_value=0)["SOC(t)"])

        ########################################################################################
        #### Energie totale (W+PV+BESS) produite
        AEP_Data["Energy Hybrid+BESS"] = ""
        AEP_Data.loc[(AEP_Data["Energy Bess"] >= 0) , "Energy Hybrid+BESS"] = AEP_Data["Energy Hybrid"] - AEP_Data["Energy Bess"] * (1/eta_ch)
        AEP_Data.loc[(AEP_Data["Energy Bess"] < 0), "Energy Hybrid+BESS"] = AEP_Data["Energy Hybrid"] + abs(AEP_Data["Energy Bess"] * eta_disch)

        return None
    
    def LCOH(mix_values):
        AEP_Data["Energy Wind ratio"] = AEP_Data["Nador Wind [516 MW]"]  / 516
        AEP_Data["Energy PV ratio"] = AEP_Data['Nador PV [300 MW]'] / 300
        AEP_Data["Energy Wind"] = ""
        AEP_Data["Energy PV"] = ""
        AEP_Data["Energy Hybrid"] = ""
        AEP_Data["Electrolysis consumption"] = ""
        AEP_Data["Energy to Electrolysis_before (MWh)"] = ""
        AEP_Data["Energy to Electrolysis_after (MWh)"] = ""
        AEP_Data["Energy Bridée"] = ""
        AEP_Data["Hydrogen Production (Kg H2)"] = ""
        
        AEP_Data["Energy Wind"] = AEP_Data["Energy Wind ratio"] * float(mix_values[0])       
        AEP_Data["Energy PV"] = AEP_Data["Energy PV ratio"] * float(mix_values[1])
        
        AEP_Data["Energy Hybrid"] = AEP_Data["Energy Wind"] + AEP_Data["Energy PV"]
                
        ### Energy After BESS        
        BESS(AEP_Data,mix_values[2],mix_values[3],0.925,0.925)
        
        AEP_Data.loc[(AEP_Data["Energy Hybrid+BESS"] > float(mix_values[2])) ,
                     "Energy to Electrolysis_before (MWh)"] =  float(mix_values[2])               
        AEP_Data.loc[(AEP_Data["Energy Hybrid+BESS"] <= float(mix_values[2])) ,
                     "Energy to Electrolysis_before (MWh)"] =  AEP_Data["Energy Hybrid+BESS"]
        
        AEP_Data.loc[AEP_Data["Energy to Electrolysis_before (MWh)"] < 0, "Energy to Electrolysis_before (MWh)"] = 0
        
        AEP_Data["Electrolysis consumption"] = (AEP_Data["Energy to Electrolysis_before (MWh)"]*25)/500
        AEP_Data["Energy to Electrolysis_after (MWh)"] = (AEP_Data["Energy to Electrolysis_before (MWh)"] - AEP_Data["Electrolysis consumption"])
                                                          
        AEP_Data["Hydrogen Production (Kg H2)"]=(AEP_Data["Energy to Electrolysis_after (MWh)"]* float(1000))/conso_spécifique
        AEP_Data["Energy Bridée"] =  AEP_Data["Energy Hybrid+BESS"] -  AEP_Data["Energy to Electrolysis_after (MWh)"]
        
        water_cost=11     ## 11 dollar/ tonne H2 produit
        
        def NPCost():
            NPC = capex_bess*mix_values[3]+ capex_w*mix_values[0] + capex_pv*mix_values[1] + capex_electrolyse*mix_values[2] + H2_input_tonne*water_cost
            for i in range (1,T+1,1):
                NPC += (opex_bess*mix_values[3]+opex_w*mix_values[0] + opex_pv*mix_values[1] + opex_electrolyse*mix_values[2] +0.02*H2_input_tonne*water_cost) /((1 + wacc)**i)
            return NPC
        
        def NPV_energyH2():
            AEP = 0
            for i in range(1,T+1,1):
                AEP += (AEP_Data["Hydrogen Production (Kg H2)"].sum()/((1+wacc)**i)) 
            return AEP
        
        return ((NPCost())*1e4) / (NPV_energyH2())

    def LCOE(mix_values):
        AEP_Data["Energy Wind ratio"] = AEP_Data["Nador Wind [516 MW]"]  / 516
        AEP_Data["Energy PV ratio"] = AEP_Data['Nador PV [300 MW]'] / 300
        AEP_Data["Energy Wind"] = ""
        AEP_Data["Energy PV"] = ""
        AEP_Data["Energy Hybrid"] = ""
        AEP_Data["Electrolysis consumption"] = ""
        AEP_Data["Energy to Electrolysis_before (MWh)"] = ""
        AEP_Data["Energy to Electrolysis_after (MWh)"] = ""
        AEP_Data["Energy Bridée"] = ""
        AEP_Data["Hydrogen Production (Kg H2)"] = ""
        
        AEP_Data["Energy Wind"] = AEP_Data["Energy Wind ratio"] * float(mix_values[0])       
        AEP_Data["Energy PV"] = AEP_Data["Energy PV ratio"] * float(mix_values[1])
        
        AEP_Data["Energy Hybrid"] = AEP_Data["Energy Wind"] + AEP_Data["Energy PV"]  
        
        ### Energy After BESS        
        BESS(AEP_Data,mix_values[2],mix_values[3],0.925,0.925)
        
        AEP_Data.loc[(AEP_Data["Energy Hybrid+BESS"] > float(mix_values[2])) ,
                     "Energy to Electrolysis_before (MWh)"] =  float(mix_values[2])               
        AEP_Data.loc[(AEP_Data["Energy Hybrid+BESS"] <= float(mix_values[2])) ,
                     "Energy to Electrolysis_before (MWh)"] =  AEP_Data["Energy Hybrid+BESS"]
        
        AEP_Data.loc[AEP_Data["Energy to Electrolysis_before (MWh)"] < 0, "Energy to Electrolysis_before (MWh)"] = 0
        
        AEP_Data["Electrolysis consumption"] = (AEP_Data["Energy to Electrolysis_before (MWh)"]*25)/500
        AEP_Data["Energy to Electrolysis_after (MWh)"] = (AEP_Data["Energy to Electrolysis_before (MWh)"] - AEP_Data["Electrolysis consumption"])
                                                          
        AEP_Data["Hydrogen Production (Kg H2)"]=(AEP_Data["Energy to Electrolysis_after (MWh)"]* float(1000))/conso_spécifique
        AEP_Data["Energy Bridée"] =  AEP_Data["Energy Hybrid+BESS"] -  AEP_Data["Energy to Electrolysis_after (MWh)"]
        
        water_cost=11     ## 11 dollar/ tonne H2 produit
        
        def NPCost1():
            NPC = capex_w*mix_values[0] + capex_pv*mix_values[1]  + capex_bess*mix_values[3]
            for i in range (1,T+1,1):
                NPC += (opex_w*mix_values[0] + opex_pv*mix_values[1] + opex_bess*mix_values[3] ) /((1 + wacc)**i)
            return NPC
        
        def NPV_energyelec():
            AEP = 0
            for i in range(1,T+1,1):
                AEP += (AEP_Data["Energy Hybrid+BESS"].sum()/((1+wacc)**i)) 
            return AEP
        
        return ((NPCost1())*1e4) / (NPV_energyelec())
    
    
    def H2_constrainte_production(H2_constrainte):
        AEP_Data["Energy Wind ratio"] = AEP_Data["Nador Wind [516 MW]"]  / 516
        AEP_Data["Energy PV ratio"] = AEP_Data['Nador PV [300 MW]'] / 300
        AEP_Data["Energy Wind"] = ""
        AEP_Data["Energy PV"] = ""
        AEP_Data["Energy Hybrid"] = ""
        AEP_Data["Electrolysis consumption"] = ""
        AEP_Data["Energy to Electrolysis_before (MWh)"] = ""
        AEP_Data["Energy to Electrolysis_after (MWh)"] = ""
        AEP_Data["Energy Bridée"] = ""
        AEP_Data["Hydrogen Production (Kg H2)"] = ""
        
        AEP_Data["Energy Wind"] = AEP_Data["Energy Wind ratio"] * float(H2_constrainte[0])       
        AEP_Data["Energy PV"] = AEP_Data["Energy PV ratio"] * float(H2_constrainte[1])
        
        AEP_Data["Energy Hybrid"] = AEP_Data["Energy Wind"] + AEP_Data["Energy PV"]  
        
        ### Energy After BESS        
        BESS(AEP_Data,H2_constrainte[2],H2_constrainte[3],0.925,0.925)
        
        AEP_Data.loc[(AEP_Data["Energy Hybrid+BESS"] > float(H2_constrainte[2])) ,
                     "Energy to Electrolysis_before (MWh)"] =  float(H2_constrainte[2])               
        AEP_Data.loc[(AEP_Data["Energy Hybrid+BESS"] <= float(H2_constrainte[2])) ,
                     "Energy to Electrolysis_before (MWh)"] =  AEP_Data["Energy Hybrid+BESS"]
        
        AEP_Data.loc[AEP_Data["Energy to Electrolysis_before (MWh)"] < 0, "Energy to Electrolysis_before (MWh)"] = 0
        
        AEP_Data["Electrolysis consumption"] = (AEP_Data["Energy to Electrolysis_before (MWh)"]*25)/500
        AEP_Data["Energy to Electrolysis_after (MWh)"] = (AEP_Data["Energy to Electrolysis_before (MWh)"] - AEP_Data["Electrolysis consumption"])
                                                          
        AEP_Data["Hydrogen Production (Kg H2)"]=(AEP_Data["Energy to Electrolysis_after (MWh)"]* float(1000))/conso_spécifique
        
        H2_produced = AEP_Data["Hydrogen Production (Kg H2)"].sum() 
        
        H2 = H2_produced - (H2_input_tonne*1000) 
        
        return H2 
    
                
    # initial guesses au hasard
    mix_values0 = np.zeros(4)
    mix_values0[0] = initial_w       ##Pw
    mix_values0[1] = initial_pv      ##Ppv
    mix_values0[2] = initial_elec    ##C_elec
    mix_values0[3] = initial_bess    ##C_elec

    # show initial objective
    print('Initial Objective: ' + str(LCOH(mix_values0)/1e4))

    # domaine de définition
    b1=(1,5000)  ##Pw 
    b2=(1,5000)  ##Ppv
    b3=(1,5000) ##C_elec
    b4=(1,5000) ##C_elec

    bnds=(b1,b2,b3,b4)
    conss={'type':'eq','fun':H2_constrainte_production}
    
    ##Appel de la fonction d'optimisation
    solution = minimize(LCOH,mix_values0,method='SLSQP',bounds=bnds,constraints=conss,options={'maxiter': 200, 'ftol': 1e-4})#options={'maxiter': 200, 'ftol': 1e-3})   ##options : nbr itérations maximale et précision
    mix_values = solution.x
    print(solution)
                      
    # print solution
    print('LCOH = : ' + str(LCOH(mix_values)/1e4))
    print('LCOE =' + str(LCOE(mix_values)/1e4))
    print('Pw = ' + str(mix_values[0]) + '(MW)' )
    print('Ppv = ' + str(mix_values[1])+ '(MW)')
    print('Pelectrolyse = ' + str(mix_values[2])+ '(MW)')
    print('C_Bess = ' + str(mix_values[3])+ '(MW)')
    
    annual_H2 = AEP_Data["Hydrogen Production (Kg H2)"].sum() / 1000
    annual_Elec_to_H2 = AEP_Data["Energy to Electrolysis_after (MWh)"].sum() 
    annual_energy_lost = AEP_Data["Energy Bridée"].sum()
    percentage_lost = AEP_Data["Energy Bridée"].sum() / (AEP_Data["Energy Hybrid"].sum() - AEP_Data["Electrolysis consumption"].sum())
    annual_Elec_consumption = AEP_Data["Electrolysis consumption"].sum() 
    Electrloyse_lf = (AEP_Data["Hydrogen Production (Kg H2)"].sum()*1000/(8760*1000)) / (mix_values[2]*1000/conso_spécifique)
    Needed_Water = AEP_Data["Hydrogen Production (Kg H2)"].sum()*11/1000

    
    print('Annual hydrogen production =' + str(annual_H2) + 'tonne')
    print('Annual Electricity to H2 =' + str(annual_Elec_to_H2) + 'MWh')
    print('% electrolyse load factor =' + str(Electrloyse_lf) + '%' )
    print('% of lost electricity =' + str(percentage_lost) + '%' )
    print('Needed_Water  =' + str(Needed_Water) + 'm3' )


    return [LCOH(mix_values)/1e4,LCOE(mix_values)/1e4,mix_values[0],mix_values[1],mix_values[2],mix_values[3],annual_Elec_to_H2,annual_H2,Electrloyse_lf,annual_Elec_consumption]                    

## DASH (celui d'un code très similaire, let me know ila mhtaj chi haja f dash)

In [ ]:
app = JupyterDash(external_stylesheets=[dbc.themes.MINTY],suppress_callback_exceptions=True)
app.layout = dbc.Container(
    [
        dbc.Row([
            dbc.Col([
                html.H2("LCOE Optimization - Baseload/H2 Production", style={'color' : 'navy', 'fontSize' : 38, 'font-weight': '600'}),
                html.H5("Draft 2"),
            ], width=True),
        ], align="end"),
        html.Hr(),  
    
        html.Hr(),
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Wind Parameters", style={'color' : 'royalblue', 'fontSize' : 25, 'font-weight': '400','font-family': 'sans-serif'}),
                    html.P("CapEx [$/MW]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='capex_w',value=1000000 , placeholder="hint (1000000 ; 1200000", type="number",min=0),
                    html.P("OpEx [$/MW/year]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='opex_w',value=22300,placeholder="hint (2.32% Capex)", type="number",min=0),
                    html.P("Min Bound Pw", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='minpw',value=0,placeholder="minimum installed capacity", type="number",min=0),
                    html.P("Max Bound Pw", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='maxpw',value=2000,placeholder="maximum installed capacity", type="number",min=0),
                ]),
            ]),
            dbc.Col([
                html.Div([
                    html.H5("Solar PV Parameters", style={'color' : 'royalblue', 'fontSize' : 25, 'font-weight': '400','font-family': 'sans-serif'}),
                    html.P("CapEx [$/MW]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='capex_pv',value=600000 , placeholder="hint (617000)" ,type="number",min=0),
                    html.P("OpEx [$/MW/year]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='opex_pv',value=16560 ,placeholder="hint (2.76% Capex)", type="number",min=0),
                    html.P("Min Bound Ppv", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='minppv',value=0,placeholder="minimum installed capacity", type="number",min=0),
                    html.P("Max Bound Ppv", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='maxppv',value=2000,placeholder="maximum installed capacity", type="number",min=0),                    
                ]),
            ]),
            dbc.Col([
                html.Div([
                    html.H5("BESS Parameters", style={'color' : 'royalblue', 'fontSize' : 25, 'font-weight': '400','font-family': 'sans-serif'}),
                    html.P("CapEx [$/MW]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='capex_bess',value=380000, placeholder="hint (383500 ; 300000)" , type="number",min=0),
                    html.P("OpEx [$/MW/year]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='opex_bess',value=19000, placeholder="hint (5% Capex including battery replacement)", type="number",min=0),
                    html.P("Charge Efficiency", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='ch_eff',value=0.925,placeholder="minimum installed capacity", type="number",min=0,max=1),
                    html.P("Discharge Efficiency", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='dis_eff',value=0.925,placeholder="maximum installed capacity", type="number",min=0,max=1),
                    html.P("Min Bound Pbess", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='minpbess',value=0,placeholder="minimum installed capacity", type="number",min=0),
                    html.P("Max Bound Pbess", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='maxpbess',value=2000,placeholder="maximum installed capacity", type="number",min=0),
                ]),
            ]),
            dbc.Col([
                html.Div([
                    html.H5("Electrolysis Parameters", style={'color' : 'royalblue', 'fontSize' : 25, 'font-weight': '400','font-family': 'sans-serif'}),
                    html.P("CapEx [$/MW]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='capex_elec',value=700000, placeholder="hint (700000;600000) + 11% + 10 %, installation/soft cost", type="number",min=0),
                    html.P("OpEx [$/MW/year]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='opex_elec', value=14000,placeholder="hint (2% Capex not including stack replacement)", type="number",min=0),
                    html.P("Specific Electricity Consumption [kWh/kg]", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='conso_spec',value=52,placeholder="hint (54 ;52 ;50)", type="number",min=33, max=100),
                    html.P("Selling price H2 [$/kg]", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='sel_price',value=1.5,placeholder="hint (1.5 ; 2)", type="number",min=0.1, max=5),
                    html.P("Min Bound Pelectrolysis", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='minpelec',value=0,placeholder="minimum installed capacity", type="number",min=0),
                    html.P("Max Bound Pelectrolysis", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='maxpelec',value=2000,placeholder="maximum installed capacity", type="number",min=0),
                ]),
            ]),
            
        ]),
        html.Hr(),
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Basification parameters", style={'color' : 'royalblue', 'fontSize' : 25, 'font-weight': '400','font-family': 'sans-serif'}),
                    html.P("Capacité Baseload [MW]:", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='cap_base',value=300, placeholder="hint (100 ; 200 ; 300)", type="number",min=0),
                    html.P("Minimum Load Factor (0.X):", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='load_factor',value=0.9,placeholder="must be between 0 and 1", type="number",min=0, max=1),      
                ]),
            ]),
            dbc.Col([
                html.Div([
                    html.H5("System parameters", style={'color' : 'royalblue', 'fontSize' : 25, 'font-weight': '400','font-family': 'sans-serif'}),
                    html.P("Wacc (0.0X):", style={'color' : 'black', 'fontSize' : 20, 'font-weight': '400'}),
                    dbc.Input(id='wacc',value=0.07,placeholder="hint (0.07)", type="number",min=0),
                    html.Hr(),
                    html.H5("Commands"),
                    dbc.Button("Run Optimization (> 13min)", id="opt", size="lg",color="success", style={"margin": "10px"},
                               n_clicks_timestamp='0'),
                    dbc.Button("Stop Optimization", id="stop", size="lg",color="warning", style={"margin": "10px"}),
#                     daq.StopButton(id='stop',buttonText='Stop Opt',style={"margin": "10px"})
#                   dbc.Button("Stop Optimization",id="cancel_button_id",color="warning", style={"margin": "10px"},
#                                n_clicks_timestamp='0'),
                ]),
            ]),
        ]),
        html.Hr(),
        html.Hr(),
        html.H2('Results'),
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("LCOE / mix capacity table"),
                    dbc.Spinner(
                    html.P(id='output'),
                    color="success",
                    type="grow")
                    ])
            ]),
            dbc.Col([
                html.Div([
                 html.H5("Annual Production"),  
                 dbc.Spinner(   
                 dcc.Graph(id='graph-output', figure={}),color="success",type="grow")
                ])
            ])
        ]),
        html.Hr(),
    ],
    fluid=True)

##Display results in Dataframe
def make_table(dataframe):
    return dbc.Table.from_dataframe(
        dataframe,
        bordered=True,
        hover=True,
        responsive=True,
        striped=True,
        style={

        }
    )
T=25
######################################## Call Back


@app.callback(
    [Output('output', 'children'),Output('graph-output', 'figure')],
    [Input("opt", 'n_clicks_timestamp'),Input("stop", 'n_clicks_timestamp')],
    [
        State('capex_w', 'value'),
        State('opex_w', 'value'),
        State('minpw', 'value'),
        State('maxpw', 'value'),
        State('capex_pv', 'value'),
        State('opex_pv', 'value'),
        State('minppv', 'value'),
        State('maxppv', 'value'),
        State('capex_bess', 'value'),
        State('opex_bess', 'value'),
        State('ch_eff', 'value'),
        State('dis_eff', 'value'),
        State('minpbess', 'value'),
        State('maxpbess', 'value'),
        State('capex_elec', 'value'),
        State('opex_elec', 'value'),
        State('conso_spec', 'value'),
        State('sel_price', 'value'),
        State('minpelec', 'value'),
        State('maxpelec', 'value'),
        State('cap_base', 'value'),
        State('load_factor', 'value'),
        State('wacc', 'value'),
    ],
    prevent_initial_call=True

)



def Run_optimization(opt,stop,capex_w,opex_w,minpw,maxpw,
                     capex_pv,opex_pv,minppv,maxppv,
                     capex_bess,opex_bess,ch_eff,dis_eff,minpbess,maxpbess,
                     capex_elec,opex_elec,conso_spec,sel_price,minpelec,maxpelec,
                     cap_base,load_factor,wacc):
          
        try:
            button_pressed = np.argmax(np.array([
                float(opt),
                float(stop),
            ]))
            assert button_pressed is not None
        except:
            button_pressed = 0

        if button_pressed == 0:

            result=get_min_LCOE(capex_w,opex_w,minpw,maxpw,
                            capex_pv,opex_pv,minppv,maxppv,
                            capex_bess,opex_bess,ch_eff,dis_eff,minpbess,maxpbess,
                            capex_elec,opex_elec,conso_spec,sel_price,minpelec,maxpelec,
                            cap_base,load_factor,wacc)
            
            output = make_table(pd.DataFrame(
                {
                    "Output": [
                        "LCOE ($/MWh)",
                        "Load Factor",
                        "Capacité éolienne (MW)",
                        "Capacité Solaire PV (MW)",
                        "Capacité BESS (MWh)",
                        "Capacité électrolyse (MW)",
                        "Curtailment (%)"
                    ],
                    "Value" : [
                        round(float(result[0]),3),
                        round(float(result[8]),4),
                        round(float(result[1]),0),
                        round(float(result[2]),0),
                        round(float(result[4]),0),
                        round(float(result[3]),0),
                        round(float(result[7]),2)
                    ]
                }
            ))
            
            test=AEP_Data.drop(['Date','Time'],axis=1)
            test=test.astype(np.float64)
            test=test.resample(rule='A').sum()
            test["Energy Baseload (GWh)"] = test["Energy Baseload"] / 1000
            test['Hydrogen Production / 100 (tonnes)'] = test["Hydrogen Production (Kg H2)"] / 100000
            test["Energy Electrolysis (GWh)"] = test["Energy Electrolysis (MWh)"] / 1000
            test["Energy Wind+PV+BESS (GWh)"] = test["Energy Hybrid+BESS"] / 1000
            
            fig = px.bar(data_frame=test, 
                      x=test.index.year, 
                      y=['Energy Wind+PV+BESS (GWh)',"Energy Baseload (GWh)","Energy Electrolysis (GWh)","Hydrogen Production / 100 (tonnes)"],
                      color_discrete_sequence =['goldenrod','orangered','navy','skyblue'],
                      template="plotly_white",
                      width=1200,
                      title="Annual Production",
                      barmode='group',
            )
            fig.update_layout(width=1200, height=500, bargap=0.3)
            fig.update_traces(width=0.2)
            fig.update_xaxes(ticks="outside",nticks=50,tickwidth=1,tickangle=60,title_text="Year",title_font=dict(color='navy'))

        elif button_pressed == 1:
             output = str('Optimization Stoped')
             #fig == {}

        return output, fig

app.run_server(debug=True,mode='external', port=8001)